In [7]:
!pip install tensorflow_datasets==4.9.3 -q


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
dataset, info = tfds.load('oxford_iiit_pet', with_info=True)